# Visualization

In [ ]:
from pathlib import Path
from typing import Optional, Sequence, Union, List

import duckdb
import pandas as pd
import folium

def query_ais_duckdb(
    root_path: Union[str, Path] = "ais-data-parquet",
    dates: Optional[Union[str, Sequence[str]]] = None,
    mmsi: Optional[Union[str, Sequence[str]]] = None,
    segments: Optional[Union[int, Sequence[int]]] = None,
    columns: Optional[Sequence[str]] = None,
    verbose: bool = False,
) -> pd.DataFrame:
    """
    Fast query helper using DuckDB to read AIS data from the partitioned
    parquet dataset generated by `ais_to_parquet`.

    Parameters
    ----------
    root_path : str or Path, optional
        Root directory of the parquet dataset (default: "ais_data_parquet").
    dates : str or list[str], optional
        Date(s) to filter on (e.g. "2025-11-05", or ["2025-11-05", "2025-11-06"]).
        If None, no date filter is applied.
    mmsi : str or list[str], optional
        MMSI or list of MMSIs to filter on. If None, no MMSI filter is applied.
    segments : int or list[int], optional
        Segment ID(s) to filter on. If None, no segment filter is applied.
    columns : list[str], optional
        Subset of columns to select. If None, selects all columns ("*").
    verbose : bool, optional
        If True, prints the generated SQL query.

    Returns
    -------
    pd.DataFrame
        Result of the DuckDB query as a pandas DataFrame.

    Examples
    --------
    >>> df = query_ais_duckdb(dates="2025-11-05")
    >>> df = query_ais_duckdb(dates="2025-11-05", mmsi="219000123")
    >>> df = query_ais_duckdb(
    ...     dates=["2025-11-05", "2025-11-06"],
    ...     mmsi=["219000123", "219000456"],
    ...     columns=["MMSI", "Timestamp", "Latitude", "Longitude"]
    ... )
    """
    root_path = Path(root_path)
    if not root_path.exists():
        raise FileNotFoundError(f"No parquet dataset found at: {root_path}")

    # Normalization helpers
    def _to_list(x) -> Optional[List]:
        if x is None:
            return None
        if isinstance(x, (str, int)):
            return [x]
        return list(x)

    def _sql_list_str(values: Sequence[str]) -> str:
        return ", ".join(f"'{v}'" for v in values)

    def _sql_list_int(values: Sequence[int]) -> str:
        return ", ".join(str(v) for v in values)

    dates_list = _to_list(dates)
    mmsi_list = _to_list(mmsi)
    segments_list = _to_list(segments)

    # Columns
    if columns is None:
        col_expr = "*"
    else:
        col_expr = ", ".join(columns)

    parquet_glob = str(root_path / "**" / "*.parquet")

    sql = f"SELECT {col_expr} FROM read_parquet('{parquet_glob}') WHERE 1=1"

    if dates_list is not None:
        sql += f" AND Date IN ({_sql_list_str([str(d) for d in dates_list])})"

    if mmsi_list is not None:
        sql += f" AND MMSI IN ({_sql_list_str([str(m) for m in mmsi_list])})"

    if segments_list is not None:
        sql += f" AND Segment IN ({_sql_list_int([int(s) for s in segments_list])})"

    if verbose:
        print("[query_ais_duckdb] SQL:\n", sql)

    con = duckdb.connect(database=":memory:")
    df = con.execute(sql).df()
    con.close()
    return df

def make_ais_folium_map(
    date: str,
    root_path: Union[str, Path] = "ais-data-parquet",
    mmsi: Optional[Union[str, Sequence[str]]] = None,
    tiles: str = "CartoDB positron",
    zoom_start: int = 8,
    verbose: bool = False,
) -> folium.Map:
    """
    Create a Folium map of AIS tracks for a given date, optionally for only a
    subset of vessels (MMSIs).

    Parameters
    ----------
    date : str
        Date to visualize (e.g. "2025-11-05").
    root_path : str or Path, optional
        Root directory of the parquet dataset (default: "ais_data_parquet").
    mmsi : str or list[str], optional
        MMSI(s) to include. If None, includes all vessels on that date.
    tiles : str, optional
        Folium tiles style (default: "CartoDB positron").
    zoom_start : int, optional
        Initial zoom level for the map.
    verbose : bool, optional
        Print some info about the data being plotted.

    Returns
    -------
    folium.Map
        A Folium map with vessel tracks drawn as polylines.

    Examples
    --------
    >>> m = make_ais_folium_map("2025-11-05")
    >>> m.save("ais_tracks_2025-11-05.html")
    >>>
    >>> m = make_ais_folium_map("2025-11-05", mmsi=["219000123", "219000456"])
    """
    # Query only what we need
    df = query_ais_duckdb(
        root_path=root_path,
        dates=date,
        mmsi=mmsi,
        columns=["MMSI", "Timestamp", "Latitude", "Longitude", "Segment"],
        verbose=verbose,
    )

    if df.empty:
        raise ValueError(f"No AIS data found for date={date} and mmsi={mmsi}")

    if verbose:
        print(
            f"[make_ais_folium_map] Loaded {len(df):,} rows, "
            f"{df['MMSI'].nunique():,} vessels"
        )

    # Ensure proper dtypes / ordering
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    df = df.sort_values(["MMSI", "Segment", "Timestamp"])

    # Map center: mean of all positions
    center_lat = df["Latitude"].mean()
    center_lon = df["Longitude"].mean()

    m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start, tiles=tiles)

    # Group by vessel (and optionally segment)
    for (mmsi_val, seg_val), g in df.groupby(["MMSI", "Segment"]):
        coords = g[["Latitude", "Longitude"]].to_numpy().tolist()
        if len(coords) < 2:
            continue  # not enough points to draw a line

        tooltip = f"MMSI: {mmsi_val}, Segment: {seg_val}"
        folium.PolyLine(
            locations=coords,
            weight=2,
            opacity=0.8,
            tooltip=tooltip,
        ).add_to(m)

    return m


In [8]:
m = make_ais_folium_map("2025-11-05", verbose=True)
m.save("ais_tracks_2025-11-05.html")

[query_ais_duckdb] SQL:
 SELECT MMSI, Timestamp, Latitude, Longitude, Segment FROM read_parquet('ais-data-parquet\**\*.parquet') WHERE 1=1 AND Date IN ('2025-11-05')
[make_ais_folium_map] Loaded 184,183 rows, 123 vessels


In [7]:
m = make_ais_folium_map(
    date="2025-11-02",
    mmsi=["219006113", "219009229"],
    verbose=True,
)
m.save("ais_tracks_subset_2025-11-05.html")


[query_ais_duckdb] SQL:
 SELECT MMSI, Timestamp, Latitude, Longitude, Segment FROM read_parquet('ais-data-parquet\**\*.parquet') WHERE 1=1 AND Date IN ('2025-11-02') AND MMSI IN ('219006113', '219009229')
[make_ais_folium_map] Loaded 18,843 rows, 2 vessels


Loaded 1 routes.
Saved map as dkcpc_lines_bbox.html
